# Rapid Construction of DKGs in ASKEM

This notebook explains what goes into creating a domain knowledge graph (DKG) and how new ones can be rapidly constructed for new use cases.


## Configuration

MIRA builds on a combination of pre-ASKEM and ASKEM-specific infrastructure for acquiring, parsing, and processing ontologies, knowledge graphs, and databases. In order to determine which parts of this infrastructure are used and how, each DKG is configured with the following pieces of information:

1. What ontologies should be pulled in? For example, the epidemiology use case brings in the Infectious Disease Ontology (IDO), the National Cancer Institute Thesaurus (NCIT), and several others from the biomedical domain.
2. Is there a custom ASKEM domain ontology? For both epidemiology and space weather, we have constructed custom domain ontologies to help align bits from existing ontologies as well as to curate novel terms that are necessary for grounding the concepts appearing in models in the domain
3. Custom knowledge resources (these are quite custom and not easily translatable to a declarative configuration).

There are several pre-defined configurations for DKGs built in to MIRA including:

- `epi` for epidemiology
- `space` for space weather
- `eco` for ecology

## Running the Constructor

After installing MIRA, the pre-defined DKGs can be constructed using the following shell script:

```shell
python -m mira.dkg.construct --use-case epi
```

This does the following:

1. Creates a nodes and edges file that can be loaded into a Neo4j graph database. These are orchestrated along with the MIRA web application in Docker (more information [here](https://github.com/indralab/mira/tree/main/docker)).
2. Creates an RDF dump that can be loaded in any triple store and queried with SPARQL
3. Uses graph machine learning to create dense node embeddings based on the topology of the knowledge graph
4. Creates a "metaregistry" configuration file. Note that we actually combine all of the metaregistry configrations together into a single service running at http://34.230.33.149:8772/.

When these have been adequately locally tested, the command can be run with the `--do-upload` flag to automate uplaoding these artifacts to S3. The uploads are versioned by the date on which the build was performed. Inside the DKG, various nodes, edges, and metadata are tagged with the version of the resources (e.g., ontologies) from which they came.

## Arbitrary Constrution

In addition to the pre-configured DKGs, new DKGs can be rapidly constructed using the Python API to MIRA.

In [1]:
from mira.dkg.construct import construct, DKGConfig

While the epidemiology use case had a close relationship between the DKG and the modeling framework, the new regnet framework being proposed in the early May 2023 hackathon can appear in several domains including:

- gene transcription regulatory networks
- ecological networks (i.e., predator-prey networks *a la* Lotka-Volterra)
- cell signalling networks

In the following example, we construct an *ad hoc* domain knowledge graph by specifying several resources useful for describing genes and phenomena that may appear in these networks including:

- HGNC - an ontology of human genes that contains relationships to gene families families, proteins. Useful for annotating components of models.
- Gene Ontology (GO) - an ontology of molecular functional and biological processes used in annotating genes. Useful for both model-level annotation and annotating relationships between components of models (e.g., edges).
- WikiPathways - a resource categorizing genes in functional pathways. Useful for model-level annotation
- ProbOnto - an ontology of probability distributions

In [ ]:
genereg_config = DKGConfig(
    prefixes=[
        "hgnc",
        "go",
        "wikipathways",
        "probonto",
    ],
)

construct(
    use_case="genreg", 
    config=genereg_config,
    # do_upload=True,  # uncomment when you're ready to send to S3!
)

Alternatively, if you would rather use the command line, you can declare this configuration  as the following JSON (let's say it's called `config.json`:

```json
{
  'use_case': 'genreg',
  'prefixes': ['hgnc', 'go', 'wikipathways', 'probonto']
}
```

Then run the following code:

```shell
python -m mira.dkg.construct --use-case config.json
```

The `--use-case` flag is smart enough to figure out if it's pointing to a file or a pre-configured name for a use case.

## Next Steps

We've built on two exsting softwares, `pyobo` and `bioontologies` for the discovery and parsing of ontologies, but we don't yet have abstract tooling for ingesting knowledge graphs.

For example, in the ecological domain (where we might have predator and prey models like Lotka-Volterra), it would be useful to get and ingest the Global Biotic Interaction (GloBI) knowledge graph, which contains interactions between agents in the environment such as:

- What do sea otters (Enhydra lutris) eat?
- What do honey bees (Apis) pollinate?

These could also provide useful constraints on model construction, such as making sure that there is no relationship where a rabbit eats a wolf.